In [1]:
import numpy as np
import pandas as pd

# Import alcohol dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-06-26/week13_alcohol_global.csv')

In [3]:
df.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 7.7+ KB


# Join dataset with two letters code for each country

In [5]:
country_short = pd.read_csv('country_short.csv')

In [6]:
country_short.head()

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [7]:
country_short

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis and Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


In [8]:
df['country'] = df['country'].str.replace('&', 'and')

In [9]:
df_2 = df.merge(country_short,
        how = 'left',
        left_on='country',
        right_on='Name').drop(columns='Name')

In [10]:
df_2.set_index('country', inplace=True)

### For countries with NAs in code, I manually imputed country codes.

In [11]:
for country in df_2[df_2['Code'].isna()].index:
    name = input(country +': ')
    df_2.at[country, 'Code'] = name

Bolivia: BO
Bosnia-Herzegovina: BA
Brunei: BN
Cote d'Ivoire: CI
Cabo Verde: CV
North Korea: KP
DR Congo: CD
Iran: IR
Laos: LA
Micronesia: FM
Namibia: NA
South Korea: KR
Moldova: MD
St. Kitts and Nevis: KN
St. Lucia: LC
St. Vincent and the Grenadines: VC
Syria: SY
Macedonia: MK
Tanzania: TZ
USA: US
Venezuela: VE
Vietnam: VN


# Join continent dataset

In [47]:
continents = pd.read_csv('continents.csv')
continents = continents[['Two_Letter_Country_Code', 'Three_Letter_Country_Code', 'Continent_Name', 'Continent_Code']]

In [48]:
continents.at[152, 'Two_Letter_Country_Code'] = 'NA'

In [49]:
df_3 = df_2.reset_index().merge(continents, 
          how = 'left',
          left_on='Code',
          right_on='Two_Letter_Country_Code').drop(columns='Two_Letter_Country_Code')

In [50]:
df_3[df_3['Continent_Name'].isna()]

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,Code,Three_Letter_Country_Code,Continent_Name,Continent_Code


In [51]:
df_3

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,Code,Three_Letter_Country_Code,Continent_Name,Continent_Code
0,Afghanistan,0,0,0,0.0,AF,AFG,Asia,AS
1,Albania,89,132,54,4.9,AL,ALB,Europe,EU
2,Algeria,25,0,14,0.7,DZ,DZA,Africa,AF
3,Andorra,245,138,312,12.4,AD,AND,Europe,EU
4,Angola,217,57,45,5.9,AO,AGO,Africa,AF
...,...,...,...,...,...,...,...,...,...
195,Venezuela,333,100,3,7.7,VE,VEN,South America,SA
196,Vietnam,111,2,1,2.0,VN,VNM,Asia,AS
197,Yemen,6,0,0,0.1,YE,YEM,Asia,AS
198,Zambia,32,19,4,2.5,ZM,ZMB,Africa,AF


# Join population dataset

I added 3 new columns:
* Population in 1998,
* Population in 2018,
* Percentage difference in population between 1998 and 2018.

In [52]:
pop = pd.read_csv('pop.csv', delimiter=';').drop(columns='Country Name')
pop.rename(columns={'2018': 'pop_2018', '1998': 'pop_1998'}, inplace=True)

In [53]:
pop['Population_diff'] = (pop['pop_2018'] - pop['pop_1998'])/ pop['pop_1998']

In [54]:
df_4 = df_3.merge(pop,
          how='left',
          left_on='Three_Letter_Country_Code',
          right_on='Country Code').drop(columns='Country Code')

# Join GDP dataset

I added 3 new columns:
* GDP in 1998,
* GDO in 2018,
* Percentage difference in GDP between 1998 and 2018.

In [55]:
gdp = pd.read_csv('gdp.csv', delimiter=';').drop(columns='Country Name')
gdp.rename(columns={'2018': 'gdp_2018', '1998': 'gdp_1998'}, inplace=True)

In [56]:
gdp

,Country Code,gdp_1998,gdp_2018
0,ABW,1.665101e+09,NaN
1,AFG,NaN,1.936297e+10
2,AGO,6.506230e+09,1.057510e+11
3,ALB,2.545967e+09,1.510250e+10
4,AND,1.211932e+09,3.236544e+09
...,...,...,...
259,XKX,NaN,7.938991e+09
260,Rep.,6.838557e+09,2.681870e+10
261,ZAF,1.377748e+11,3.682889e+11
262,ZMB,3.537683e+09,2.672007e+10


In [57]:
gdp['gdp_1998']

0      1.665101e+09
1               NaN
2      6.506230e+09
3      2.545967e+09
4      1.211932e+09
           ...     
259             NaN
260    6.838557e+09
261    1.377748e+11
262    3.537683e+09
263    6.401968e+09
Name: gdp_1998, Length: 264, dtype: float64

In [58]:
gdp['GDP_diff'] = (gdp['gdp_2018'] - gdp['gdp_1998'])/ gdp['gdp_1998']

In [59]:
df_5 = df_4.merge(gdp,
          how='left',
          left_on='Three_Letter_Country_Code',
          right_on='Country Code').drop(columns='Country Code')

# Join sub-region dataset

In [60]:
region = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')
region = region[['alpha-3', 'sub-region']]

In [61]:
df_6 = df_5.merge(region,
          how='left',
          left_on='Three_Letter_Country_Code',
          right_on='alpha-3').drop(columns='alpha-3')

# Join religion dataset

As religion I have chosen the most popular religion in the country.

In [62]:
religion = pd.read_excel('religion.xlsx')
religion = religion[religion['Year']==2010]

In [63]:
religion.drop(columns=['row_number', 'level', 'Nation_fk', 'Year', 'Region', 'All Religions'])

,Country,Christians,Muslims,Unaffiliated,Hindus,Buddhists,Folk Religions,Other Religions,Jews
0,All Countries,"2,168,330,000","1,599,700,000","1,131,150,000","1,032,210,000","487,760,000","404,690,000","58,150,000","13,860,000"
1,All Countries,"266,630,000","3,480,000","59,040,000","2,250,000","3,860,000","1,020,000","2,200,000","6,040,000"
2,All Countries,"531,280,000","840,000","45,390,000","660,000","410,000","10,040,000","990,000","470,000"
3,All Countries,"553,280,000","43,470,000","139,890,000","1,380,000","1,350,000","870,000","890,000","1,420,000"
4,All Countries,"12,710,000","317,070,000","2,100,000","1,720,000","500,000","1,060,000","230,000","5,630,000"
...,...,...,...,...,...,...,...,...,...
236,Zimbabwe,"10,930,000","110,000","990,000","<10,000","<10,000","480,000","40,000","10,000"
237,South Sudan,"6,010,000","610,000","50,000","<10,000","<10,000","3,270,000","<10,000","<10,000"
238,Curacao,"140,000","<10,000","<10,000","<10,000","<10,000","<10,000","<10,000","<10,000"
239,Sint Maarten,"40,000","<10,000","<10,000","<10,000","<10,000","<10,000","<10,000","<10,000"


In [64]:
religion = religion.melt(id_vars = ['Country'], value_vars=['Christians', 'Muslims',
                                                'Unaffiliated', 'Hindus',
                                                'Buddhists', 'Folk Religions',
                                                'Other Religions', 'Jews'],
             var_name='Religion')

In [65]:
religion['value'] = religion['value'].str.replace('<', '').str.replace(',', '').astype(float)

In [66]:
religion = religion.groupby('Country').apply(lambda country: country.nlargest(1, columns='value')).reset_index(drop=True)

In [67]:
df_7 = df_6.merge(religion,
          how='left',
          left_on='country',
          right_on='Country').drop(columns=['Country', 'value'])

For countries without religion I have manually imputed informations.

In [68]:
df_7.set_index('country', inplace=True)
for country in df_7[df_7['Religion'].isna()].index:
    name = input(country +': ')
    df_7.at[country, 'Religion'] = name

Cote d'Ivoire: Christians
Cabo Verde: Christians
Congo: Christians
DR Congo: Christians
Micronesia: Christians
Myanmar: Buddhists
Russian Federation: Christians
Russian Federation: Christians
Macedonia: Christians
USA: Christians


In [69]:
df_7.drop_duplicates(inplace=True)

In [70]:
df_7.reset_index(inplace=True)

In [71]:
df_7.country.value_counts()

Cyprus                   2
Azerbaijan               2
Russian Federation       2
Turkey                   2
Kazakhstan               2
                        ..
Sao Tome and Principe    1
Luxembourg               1
Tajikistan               1
Turkmenistan             1
Cabo Verde               1
Name: country, Length: 193, dtype: int64

In [73]:
df_final = df_7.groupby('country').apply(lambda x: x.nlargest(1, columns = 'Population_diff'))

# Join geo localization dataset

In [74]:
geo_loc = pd.read_html('https://developers.google.com/public-data/docs/canonical/countries_csv')[0]

In [75]:
geo_loc

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [77]:
df_final.columns

Index(['country', 'beer_servings', 'spirit_servings', 'wine_servings',
       'total_litres_of_pure_alcohol', 'Code', 'Three_Letter_Country_Code',
       'Continent_Name', 'Continent_Code', 'pop_1998', 'pop_2018',
       'Population_diff', 'gdp_1998', 'gdp_2018', 'GDP_diff', 'sub-region',
       'Religion'],
      dtype='object')

In [79]:
df_final = df_final.merge(geo_loc,
              how='left',
              left_on='Code',
              right_on='country').drop(columns=['name', 'country_y']).rename(columns={'country_x':'country'})

# Save to csv

In [80]:
df_final.to_csv('final_data.csv')